In [1]:
from mlxtend.regressor import StackingCVRegressor
import sys
 
sys.path.append('C:\\Users\\Nikhil\\Data_Science_Projects')
from common_ds_modules import missing_values, data_manipulation, modeling, eda
import os
import pandas as pd
import numpy as np
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
import pickle
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor

from sklearn.compose import make_column_transformer 
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import FunctionTransformer
log_transformer = FunctionTransformer(np.log1p)

from sklearn.compose import make_column_selector as selector
from scipy.stats import skew

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer

C:\Users\Nikhil\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def get_columns(df, query=''):
    return [c for c in df.columns if query.lower() in c.lower()]

In [3]:
def get_pipeline_cross_val_score(preprocessor, model, param_grid, x_train, y_train, max_models):
    print(f'Model name: {model.__name__}')
    if model.__name__ == 'Lasso' or model.__name__ == 'Ridge' or model.__name__ == 'ElasticNet':
        print(f'Scaling Pipe: {model.__name__}')
        pipe = make_pipeline(preprocessor, StandardScaler(), model())
    else:
        pipe = make_pipeline(preprocessor, model())

    if max_models is not None:
        random_search = RandomizedSearchCV(pipe, param_grid, scoring='neg_root_mean_squared_error',
                                   n_iter=max_models, verbose=2)
    else:
        random_search = GridSearchCV(pipe, param_grid, scoring='neg_root_mean_squared_error',
                                   n_iter=max_models, verbose=2)

    random_search = random_search.fit(x_train, y_train)
    best_params = { k.split('__')[1]:v for (k,v) in zip(list(random_search.best_params_.keys()), list(random_search.best_params_.values()))} 
    if model.__name__ == 'Lasso' or model.__name__ == 'Ridge' or model.__name__ == 'ElasticNet':
        print(f'Scaling Final Pipe: {model.__name__}')
        final_pipe = final_pipe = make_pipeline(preprocessor, StandardScaler(), model(**best_params))
    else:
        final_pipe = make_pipeline(preprocessor, model(**best_params))
    
    score = -cross_val_score(final_pipe, x_train, y_train, cv=5, scoring='neg_root_mean_squared_error').mean()
    
    return final_pipe, score, best_params

In [4]:
xgb_param_grid = {
    'xgbregressor__n_estimators': [110, 120],
    'xgbregressor__max_depth':[5,6],
    'xgbregressor__max_leaves':[40, 30, 50],
    'xgbregressor__reg_alpha': [i/100 for i in range(1,3)],
    'xgbregressor__reg_lambda': [i/100 for i in range(1,3)],
    'xgbregressor__colsample_bytree': [i/10 for i in range(2, 5)],
    'xgbregressor__learning_rate': [i/100 for i in range(3,6)],
    'xgbregressor__gamma': [i/100 for i in range(1, 3)]
}

lasso_param_grid = {
    'lasso__alpha':[0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 75, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
}

ridge_param_grid = {
    'ridge__alpha':[0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 75, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
}

elastic_net_param_grid = {
    'elasticnet__alpha': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 75, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    'elasticnet__l1_ratio':[0.01, 0.05, 0.1, 0.5, 1]
}

knn_param_grid = {
    'kneighborsregressor__weights' : ['uniform'],
    'kneighborsregressor__algorithm' : ['auto'],
    'kneighborsregressor__n_neighbors' : [5,10,15, 20, 25, 30],
    'kneighborsregressor__leaf_size': [10],
    'kneighborsregressor__p': [1],
}

In [5]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')


train_df['MSZoning'] = train_df['MSZoning'].apply(str)
train_df['MSZoning'] = train_df['MSZoning'].replace({'C (all)': 'C'})


test_df['MSZoning'] = test_df['MSZoning'].apply(str)
test_df['MSZoning'] = test_df['MSZoning'].replace({'C (all)': 'C'})

train_df['MSSubClass'] = train_df['MSSubClass'].apply(lambda x: str(x))
test_df['MSSubClass'] = test_df['MSSubClass'].apply(lambda x: str(x))
train_df['LogSalePrice'] = np.log1p(train_df['SalePrice'])

In [6]:
train_drop = missing_values.get_high_missing_value_columns(train_df, 30)
test_drop = missing_values.get_high_missing_value_columns(test_df, 30)


train_low_missing_vals_df = train_df.drop(train_drop, axis='columns')
test_low_missing_vals_df = test_df.drop(test_drop, axis='columns')

In [7]:
numerical_variables, categorical_variables = data_manipulation.get_numerical_categorical_variables(train_low_missing_vals_df,\
                                                                                            test_low_missing_vals_df, 'Id')
                                                                                                   

In [8]:
def plot_variable_dist(df, variables):
    for c in variables:
        print(f'Variable: {c}')
        display(df[c].describe())
        plt.hist(df[c])
        plt.title(f'Distribution for {c}')
        plt.show()
        
def plot_discrete_variable_dist(df, variables):
    for c in variables:
        print(f'Variable: {c}')
        display(df[c].value_counts(normalize=True))
        plt.hist(df[c])
        plt.title(f'Distribution for {c}')
        plt.show()
        
def plot_cat_variable_dist(df, variables):
    for c in variables:
        print(f'Variable: {c}')
        display(df[c].value_counts(normalize=True))
        plt.bar(df[c].value_counts().index, df[c].value_counts())
        plt.title(f'Distribution for {c}')
        plt.show()

In [9]:
label = 'SalePrice'

In [10]:
train_low_missing_vals_df2 = train_low_missing_vals_df.copy()
test_low_missing_vals_df2 = test_low_missing_vals_df.copy()

In [11]:
for f in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    #print(f'Variable: {f}, unique values: {train_df[f].unique()}')
    train_low_missing_vals_df2[f] = train_low_missing_vals_df2[f].fillna(value='NA')
    test_low_missing_vals_df2[f] = test_low_missing_vals_df2[f].fillna(value='NA')

In [12]:
for f in ['GarageYrBlt']:
    print(f'Variable: {f}, unique values: {train_df[f].unique()}')
    train_low_missing_vals_df2[f] = train_low_missing_vals_df2[f].fillna(value=0)
    test_low_missing_vals_df2[f] = test_low_missing_vals_df2[f].fillna(value=0)

Variable: GarageYrBlt, unique values: [2003. 1976. 2001. 1998. 2000. 1993. 2004. 1973. 1931. 1939. 1965. 2005.
 1962. 2006. 1960. 1991. 1970. 1967. 1958. 1930. 2002. 1968. 2007. 2008.
 1957. 1920. 1966. 1959. 1995. 1954. 1953.   nan 1983. 1977. 1997. 1985.
 1963. 1981. 1964. 1999. 1935. 1990. 1945. 1987. 1989. 1915. 1956. 1948.
 1974. 2009. 1950. 1961. 1921. 1900. 1979. 1951. 1969. 1936. 1975. 1971.
 1923. 1984. 1926. 1955. 1986. 1988. 1916. 1932. 1972. 1918. 1980. 1924.
 1996. 1940. 1949. 1994. 1910. 1978. 1982. 1992. 1925. 1941. 2010. 1927.
 1947. 1937. 1942. 1938. 1952. 1928. 1922. 1934. 1906. 1914. 1946. 1908.
 1929. 1933.]


In [13]:
bsmt_cols = get_columns(train_low_missing_vals_df, 'bsmt')

In [14]:
train_low_missing_vals_df2.loc[948, 'BsmtExposure'] = 'NA'
train_low_missing_vals_df2.loc[332, 'BsmtFinType2'] = 'Unf'

In [15]:
bsmt_missing_cols = ['BsmtFinType1', 'BsmtFinType2', 'BsmtExposure', 'BsmtQual', 'BsmtCond']
for f in bsmt_missing_cols:
    #print(f'Variable: {f}, unique values: {train_df[f].unique()}')
    train_low_missing_vals_df2[f] = train_low_missing_vals_df2[f].fillna(value='NA')
    test_low_missing_vals_df2[f] = test_low_missing_vals_df2[f].fillna(value='NA')

In [16]:
def fill_garage(row):
    #print(f'House built type: {type(house_built_date)}, value: {house_built_date}')
    if pd.isnull(row['GarageYrBlt']):
        if row['YearBuilt'] < 1940:
            return row['YearBuilt'] + 45
        else:
            return row['YearBuilt']
    else:
        return row['GarageYrBlt']

In [17]:
numerical_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')

categorical_variables, discrete_numerical_variables,\
continuous_numerical_variables = data_manipulation.get_variables(train_low_missing_vals_df2, [], 'Id')
discrete_numerical_variables = list(set(discrete_numerical_variables) - set(['PoolArea']))
continuous_numerical_variables = continuous_numerical_variables + ['PoolArea']
ind_continuous_variables = list(set(continuous_numerical_variables) - set(['SalePrice', 'LogSalePrice']))
numerical_variables = continuous_numerical_variables + discrete_numerical_variables

categorical_columns_selector = selector(dtype_exclude='number')
categorical_columns = categorical_columns_selector(train_low_missing_vals_df2)

ohe2 = OneHotEncoder(handle_unknown='ignore', sparse=False)
categorical_pipeline2 = make_pipeline(categorical_imputer, ohe2)



preprocessor2 = make_column_transformer((categorical_pipeline2, categorical_columns),
                                       (numerical_imputer, ind_continuous_variables),
                                       (categorical_imputer, discrete_numerical_variables),
                                    )

In [18]:
y_train = train_low_missing_vals_df2['LogSalePrice']

In [19]:
def get_comb(param_grid):
    num_combinations = 1
    for k, param_comb in xgb_param_grid.items():
        num_combinations *= len(param_comb)
        
    return num_combinations

In [20]:
get_comb(xgb_param_grid)

864

In [21]:
%%time
xgb_final_pipe, xgb_score, xgb_best_params = get_pipeline_cross_val_score(preprocessor2, XGBRegressor, xgb_param_grid, train_low_missing_vals_df2, y_train, 200)
lasso_final_pipe, lasso_score, lasso_best_params = get_pipeline_cross_val_score(preprocessor2, Lasso, lasso_param_grid, train_low_missing_vals_df2, y_train, 200)
ridge_final_pipe, ridge_score, ridge_best_params = get_pipeline_cross_val_score(preprocessor2, Ridge, ridge_param_grid, train_low_missing_vals_df2, y_train, 200)
elastic_net_final_pipe, elastic_net_score, elastic_net_best_params = get_pipeline_cross_val_score(preprocessor2, ElasticNet, elastic_net_param_grid, train_low_missing_vals_df2, y_train, 200)
knn_final_pipe, knn_score, knn_best_params = get_pipeline_cross_val_score(preprocessor2, KNeighborsRegressor, knn_param_grid, train_low_missing_vals_df2, y_train, 200)

Model name: XGBRegressor
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=6, xgbregressor__max_leaves=40, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.01, xgbregressor__reg_lambda=0.02; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=6, xgbregressor__max_leaves=40, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.01, xgbregressor__reg_lambda=0.02; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=6, xgbregressor__max_leaves=40, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.01, xgbregressor__reg_lambda=0.02; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.02, xgbregressor__learnin

[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=6, xgbregressor__max_leaves=50, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.02; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=6, xgbregressor__max_leaves=50, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.02; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=6, xgbregressor__max_leaves=50, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.02; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=6, xgbregressor__max_leaves=50, xgbregressor__n_esti

[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.01, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=6, xgbregressor__max_leaves=30, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.01, xgbregressor__reg_lambda=0.01; total time=   0.3s
[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.01, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=6, xgbregressor__max_leaves=30, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.01, xgbregressor__reg_lambda=0.01; total time=   0.3s
[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.01, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=6, xgbregressor__max_leaves=30, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.01, xgbregressor__reg_lambda=0.01; total time=   0.3s
[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.01, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=6, xgbregressor__max_leaves=30, xgbregressor__n_esti

[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.01, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=6, xgbregressor__max_leaves=30, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.02; total time=   0.3s
[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.01, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=6, xgbregressor__max_leaves=30, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.02; total time=   0.3s
[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.01, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=6, xgbregressor__max_leaves=30, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.02; total time=   0.3s
[CV] END xgbregressor__colsample_bytree=0.2, xgbregressor__gamma=0.01, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=5, xgbregressor__max_leaves=50, xgbregressor__n_esti

[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=5, xgbregressor__max_leaves=40, xgbregressor__n_estimators=120, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.01; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=5, xgbregressor__max_leaves=40, xgbregressor__n_estimators=120, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.01; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=6, xgbregressor__max_leaves=40, xgbregressor__n_estimators=120, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.02; total time=   0.3s
[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=6, xgbregressor__max_leaves=40, xgbregressor__n_esti

...

[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.1; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.1; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.1; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.1; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.1; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END .......elasticnet__alpha=75, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END .......elasticnet__alpha=75, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END .......elasticnet__

[CV] END ....elasticnet__alpha=400, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END ....elasticnet__alpha=400, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END ....elasticnet__alpha=400, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END ......elasticnet__alpha=400, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ......elasticnet__alpha=400, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ......elasticnet__alpha=400, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ......elasticnet__alpha=400, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ......elasticnet__alpha=400, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ...elasticnet__alpha=500, elasticnet__l1_ratio=0.01; total time=   0.0s
[CV] END ...elasticnet__alpha=500, elasticnet__l1_ratio=0.01; total time=   0.0s
[CV] END ...elasticnet__alpha=500, elasticnet__l1_ratio=0.01; total time=   0.0s
[CV] END ...elasticnet__alpha=500, elasticnet__l1_ratio=0.01; total time=   0.0s
[CV] END ...elasticnet__alph

[CV] END ......elasticnet__alpha=800, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ......elasticnet__alpha=800, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.01; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.01; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.01; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.01; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.01; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.05; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.05; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.05; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.05; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.05; total time=   0.0s
[CV] END ....elasticnet__alp

C:\Users\Nikhil\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=200. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV] END kneighborsregressor__algorithm=auto, kneighborsregressor__leaf_size=10, kneighborsregressor__n_neighbors=5, kneighborsregressor__p=1, kneighborsregressor__weights=uniform; total time=   0.0s
[CV] END kneighborsregressor__algorithm=auto, kneighborsregressor__leaf_size=10, kneighborsregressor__n_neighbors=5, kneighborsregressor__p=1, kneighborsregressor__weights=uniform; total time=   0.0s
[CV] END kneighborsregressor__algorithm=auto, kneighborsregressor__leaf_size=10, kneighborsregressor__n_neighbors=5, kneighborsregressor__p=1, kneighborsregressor__weights=uniform; total time=   0.0s
[CV] END kneighborsregressor__algorithm=auto, kneighborsregressor__leaf_size=10, kneighborsregressor__n_neighbors=5, kneighborsregressor__p=1, kneighborsregressor__weights=uniform; total time=   0.0s
[CV] END kneighborsregressor__algorithm=auto, kneighborsregressor__leaf_size=10, kneighborsregressor__n_neighbors=10, kneighborsregressor__p=1, kneighborsregressor__weights=uniform; total time=   0.0s

In [22]:
%%time

lr_final_pipe = make_pipeline(preprocessor2, LinearRegression())
rf_final_pipe = make_pipeline(preprocessor2, RandomForestRegressor())

estimators = [
    ('LinearRegression', lr_final_pipe),
    ('RandomForestRegressor', rf_final_pipe),
 ('KNN', knn_final_pipe),
    ('Lasso', lasso_final_pipe),
    ('Ridge', ridge_final_pipe),
    ('Elastic', elastic_net_final_pipe),
    ('XGBRegressor', xgb_final_pipe)
]


final_estimator=RandomForestRegressor()

rf_stacking_regressor = StackingRegressor(
        estimators=estimators,
        final_estimator=final_estimator,
    )

-cross_val_score(rf_stacking_regressor, train_low_missing_vals_df2, y_train, cv=5, scoring='neg_root_mean_squared_error').mean()
# 0.12747669303755846

CPU times: total: 1min 43s
Wall time: 56.1 s


0.12500337708618697

In [23]:
xgb_best_params

{'reg_lambda': 0.02,
 'reg_alpha': 0.01,
 'n_estimators': 120,
 'max_leaves': 40,
 'max_depth': 5,
 'learning_rate': 0.05,
 'gamma': 0.02,
 'colsample_bytree': 0.4}

In [24]:
rf_stacking_regressor = rf_stacking_regressor.fit(train_low_missing_vals_df2, y_train)
test_pred = rf_stacking_regressor.predict(test_low_missing_vals_df2)
test_pred = np.expm1(test_pred)
test_df['PredictedSalePrice'] = test_pred
pred = test_df[['Id', 'PredictedSalePrice']]
columns = dict()
columns['PredictedSalePrice'] = 'SalePrice'
pred = pred.rename(columns=columns)
pred.to_csv('stacking_regressor_rf_best_model_v1_6.csv', index=False)

#### Feature Engineering

In [25]:
def calc_quality_features(df):
    # calculating quality features
    df['BsmtCond'] = df['BsmtCond'].replace({"Ex":5, 'Gd':4, 'TA':3, 'Fa': 2, 'Po':1, 'NA':0})
    df['BsmtQual'] = df['BsmtQual'].replace({"Ex":5, 'Gd':4, 'TA':3, 'Fa': 2, 'Po':1, 'NA':0})
    df['ExterCond'] = df['ExterCond'].replace({"Ex":5, 'Gd':4, 'TA':3, 'Fa': 2, 'Po':1, 'NA':0})
    df['ExterQual'] = df['ExterQual'].replace({"Ex":5, 'Gd':4, 'TA':3, 'Fa': 2, 'Po':1, 'NA':0})
    df['GarageQual'] = df['GarageQual'].replace({"Ex":5, 'Gd':4, 'TA':3, 'Fa': 2, 'Po':1, 'NA':0})
    df['GarageCond'] = df['GarageCond'].replace({"Ex":5, 'Gd':4, 'TA':3, 'Fa': 2, 'Po':1, 'NA':0})
    return df

In [26]:
def calc_house_feat1(df):
    df['total_house_size'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
    df['total_baths'] = df['BsmtFullBath'] + 0.5*df['BsmtHalfBath'] + df['FullBath'] + 0.5*df['HalfBath']
    df['overall_value'] = (df['OverallCond'] + df['OverallQual'])/2
    df['garage_value'] = (df['GarageCond'] + df['GarageQual'])/2
    df['bsmt_value'] = (df['BsmtCond'] + df['BsmtQual'])/2
    df['exter_value'] = (df['ExterCond'] + df['ExterQual'])/2
    df['porch_size'] = df['OpenPorchSF'] + df['EnclosedPorch'] + df['3SsnPorch'] + df['ScreenPorch']

    df['house_age'] = df['YrSold'] - df['YearBuilt']
    df['has_second_floor'] = df.apply(lambda row: 1 if row['2ndFlrSF'] > 0 else 0 ,\
                                                  axis='columns')
    df['has_bsmt'] = df.apply(lambda row: 1 if row['TotalBsmtSF'] > 0 else 0 ,\
                                                  axis='columns')
    df['has_porch'] = df.apply(lambda row: 1 if row['porch_size'] > 0 else 0 ,\
                                                  axis='columns')
    df['bsmt_fin_ratio'] = 1 - (df['BsmtUnfSF']/df['TotalBsmtSF'])
    df['is_house_remod'] = df.apply(lambda row: 1 if row['YearBuilt'] != row['YearRemodAdd'] else 0 ,\
                                                  axis='columns')
    df['remod_wait'] = df['YearRemodAdd'] - df['YearBuilt']
    df['remod_age'] = df['YrSold'] - df['YearRemodAdd']
    
    return df

In [27]:
train_model_data2 = calc_house_feat1(calc_quality_features(train_low_missing_vals_df2))
test_model_data2 = calc_house_feat1(calc_quality_features(test_low_missing_vals_df2))

In [28]:
%%time
xgb_final_pipe, xgb_score, xgb_best_params = get_pipeline_cross_val_score(preprocessor2, XGBRegressor, xgb_param_grid, train_model_data2, y_train, 200)
lasso_final_pipe, lasso_score, lasso_best_params = get_pipeline_cross_val_score(preprocessor2, Lasso, lasso_param_grid, train_model_data2, y_train, 200)
ridge_final_pipe, ridge_score, ridge_best_params = get_pipeline_cross_val_score(preprocessor2, Ridge, ridge_param_grid, train_model_data2, y_train, 200)
elastic_net_final_pipe, elastic_net_score, elastic_net_best_params = get_pipeline_cross_val_score(preprocessor2, ElasticNet, elastic_net_param_grid, train_model_data2, y_train, 200)
knn_final_pipe, knn_score, knn_best_params = get_pipeline_cross_val_score(preprocessor2, KNeighborsRegressor, knn_param_grid, train_model_data2, y_train, 200)

Model name: XGBRegressor
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=5, xgbregressor__max_leaves=40, xgbregressor__n_estimators=120, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.02; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=5, xgbregressor__max_leaves=40, xgbregressor__n_estimators=120, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.02; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=5, xgbregressor__max_leaves=40, xgbregressor__n_estimators=120, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.02; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.02, xgbregressor__learnin

[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.05, xgbregressor__max_depth=5, xgbregressor__max_leaves=50, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.02; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.05, xgbregressor__max_depth=5, xgbregressor__max_leaves=50, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.02; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.05, xgbregressor__max_depth=5, xgbregressor__max_leaves=50, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.02; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.05, xgbregressor__max_depth=5, xgbregressor__max_leaves=50, xgbregressor__n_esti

[CV] END xgbregressor__colsample_bytree=0.2, xgbregressor__gamma=0.01, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=6, xgbregressor__max_leaves=50, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.01, xgbregressor__reg_lambda=0.01; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.2, xgbregressor__gamma=0.01, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=6, xgbregressor__max_leaves=50, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.01, xgbregressor__reg_lambda=0.01; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.2, xgbregressor__gamma=0.01, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=6, xgbregressor__max_leaves=50, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.01, xgbregressor__reg_lambda=0.01; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.2, xgbregressor__gamma=0.01, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=6, xgbregressor__max_leaves=50, xgbregressor__n_esti

[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=5, xgbregressor__max_leaves=40, xgbregressor__n_estimators=120, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.01; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=5, xgbregressor__max_leaves=40, xgbregressor__n_estimators=120, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.01; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=5, xgbregressor__max_leaves=40, xgbregressor__n_estimators=120, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.01; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.4, xgbregressor__gamma=0.01, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=5, xgbregressor__max_leaves=50, xgbregressor__n_esti

[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=6, xgbregressor__max_leaves=30, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.01, xgbregressor__reg_lambda=0.01; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.02, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=6, xgbregressor__max_leaves=30, xgbregressor__n_estimators=110, xgbregressor__reg_alpha=0.01, xgbregressor__reg_lambda=0.01; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.01, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=6, xgbregressor__max_leaves=50, xgbregressor__n_estimators=120, xgbregressor__reg_alpha=0.02, xgbregressor__reg_lambda=0.01; total time=   0.2s
[CV] END xgbregressor__colsample_bytree=0.3, xgbregressor__gamma=0.01, xgbregressor__learning_rate=0.04, xgbregressor__max_depth=6, xgbregressor__max_leaves=50, xgbregressor__n_esti

...

[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.1; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.1; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.1; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.1; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.1; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END .....elasticnet__alpha=75, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END .......elasticnet__alpha=75, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END .......elasticnet__alpha=75, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END .......elasticnet__

[CV] END ....elasticnet__alpha=400, elasticnet__l1_ratio=0.1; total time=   0.0s
[CV] END ....elasticnet__alpha=400, elasticnet__l1_ratio=0.1; total time=   0.0s
[CV] END ....elasticnet__alpha=400, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END ....elasticnet__alpha=400, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END ....elasticnet__alpha=400, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END ....elasticnet__alpha=400, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END ....elasticnet__alpha=400, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END ......elasticnet__alpha=400, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ......elasticnet__alpha=400, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ......elasticnet__alpha=400, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ......elasticnet__alpha=400, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ......elasticnet__alpha=400, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ...elasticnet__alph

[CV] END ....elasticnet__alpha=800, elasticnet__l1_ratio=0.5; total time=   0.0s
[CV] END ......elasticnet__alpha=800, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ......elasticnet__alpha=800, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ......elasticnet__alpha=800, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ......elasticnet__alpha=800, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ......elasticnet__alpha=800, elasticnet__l1_ratio=1; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.01; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.01; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.01; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.01; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.01; total time=   0.0s
[CV] END ...elasticnet__alpha=900, elasticnet__l1_ratio=0.05; total time=   0.0s
[CV] END ...elasticnet__alph

C:\Users\Nikhil\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=200. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV] END kneighborsregressor__algorithm=auto, kneighborsregressor__leaf_size=10, kneighborsregressor__n_neighbors=5, kneighborsregressor__p=1, kneighborsregressor__weights=uniform; total time=   0.0s
[CV] END kneighborsregressor__algorithm=auto, kneighborsregressor__leaf_size=10, kneighborsregressor__n_neighbors=5, kneighborsregressor__p=1, kneighborsregressor__weights=uniform; total time=   0.0s
[CV] END kneighborsregressor__algorithm=auto, kneighborsregressor__leaf_size=10, kneighborsregressor__n_neighbors=5, kneighborsregressor__p=1, kneighborsregressor__weights=uniform; total time=   0.0s
[CV] END kneighborsregressor__algorithm=auto, kneighborsregressor__leaf_size=10, kneighborsregressor__n_neighbors=10, kneighborsregressor__p=1, kneighborsregressor__weights=uniform; total time=   0.0s
[CV] END kneighborsregressor__algorithm=auto, kneighborsregressor__leaf_size=10, kneighborsregressor__n_neighbors=10, kneighborsregressor__p=1, kneighborsregressor__weights=uniform; total time=   0.0

In [29]:
rf_stacking_regressor = rf_stacking_regressor.fit(train_model_data2, y_train)
test_pred = rf_stacking_regressor.predict(test_model_data2)
test_pred = np.expm1(test_pred)
test_df['PredictedSalePrice'] = test_pred
pred = test_df[['Id', 'PredictedSalePrice']]
columns = dict()
columns['PredictedSalePrice'] = 'SalePrice'
pred = pred.rename(columns=columns)
pred.to_csv('stacking_regressor_rf_best_model_feat_eng1_v2.csv', index=False)

### Drop redundant columns

In [ ]:
redundant_columns = ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'BsmtFullBath', 'BsmtHalfBath', 'HalfBath', 'FullBath',\
                     'OverallCond', 'OverallQual', 'GarageCond', 'GarageQual', '']

In [ ]:
train_model_data3 = calc_house_feat1(calc_quality_features(train_low_missing_vals_df2))
test_model_data3 = calc_house_feat1(calc_quality_features(test_low_missing_vals_df2))